In [ ]:
import pandas as pd
import json
import time
import random
import copy

with open("test_data.json", "r") as f:
    test_data = json.loads(f.read())
df = pd.DataFrame(test_data)

def random_date():
    start = time.mktime(time.strptime("2015-01-01 00:00:00", "%Y-%m-%d %H:%M:%S"))
    end = time.mktime(time.strptime("2026-01-01 00:00:00", "%Y-%m-%d %H:%M:%S"))
    random_date = start + random.random() * (end - start)
    return time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(random_date))

random_dates = [random_date() for x in range(len(df["index"]))]
df["timestamp"] = random_dates

df.dropna()
df2 = copy.copy(df)
df3 = pd.concat([df, df2])

with open("data2.csv", "w") as f:
    df3.to_csv(f, index=False)

In [ ]:
# Generate dummy data
import datetime 
import random

def generate_data(amt_days=30, daily_samples=144):
    data = {
        "index": [x for x in range(amt_days*144)],
        "timestamp": [],
        "temperature": []
    }
    
    cur_date = datetime.datetime(2015, 3, 1, 0, 0, 0)
    step_minutes = (24*60)/daily_samples
    time_to_add = datetime.timedelta(minutes=step_minutes)

    for i in data["index"]:
        data["timestamp"].append(cur_date)
        data["temperature"].append(random.random())
        cur_date += time_to_add

    print(f"index: {len(data["index"])}, timestamp: {len(data["timestamp"])}, temperature: {len(data["temperature"])}")
    return data


In [14]:
data = generate_data()
df = pd.DataFrame(data)

with open("data4.csv", "w", encoding="utf-8") as f:
    df.to_csv(f, index=False)

index: 4320, timestamp: 4320, temperature: 4320


In [1]:
from tools.processing import DataHandler

handler = DataHandler(None)
handler.resample_csv_file("dummy_concat.csv", "resampled_test.csv")